# Nettoyage des données

## Présentation du jeu de données

L'ensemble des données utilisées ici est accessbile librement et gratuitement sur le site www.openfoodfacts.org, sous licence "Open Database Licence".
La base de données pèse en totalité 2,22Go et représente "une base de données sur les produits alimentaires faite par tout le monde, pour tout le monde".
On y trouve donc notamment les ingrédients et la composition nutritionnelle de produits trouvables en super marché, partout dans le monde, renseignés par les utilisateurs d'une application mobile. Il y a donc des imperfections : des champs sont souvent manquants ou mal renseignés.
La base de données contient 1156720 produits et chaque produit possède 178 features.

Importation des extensions utilisées dans le Notebook

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from wordcloud import WordCloud
pd.set_option("display.max_rows",200)
sns.set(font_scale=3, rc={'figure.figsize':(15,15)})
from pandas.api.types import CategoricalDtype

Récupération des données

In [2]:
data = pd.read_csv('en.openfoodfacts.org.products.csv', delimiter='\t', low_memory=False)

In [3]:
data.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1156720 entries, 0 to 1156719
Data columns (total 178 columns):
 #   Column                                      Non-Null Count    Dtype  
---  ------                                      --------------    -----  
 0   code                                        1156720 non-null  object 
 1   url                                         1156720 non-null  object 
 2   creator                                     1156716 non-null  object 
 3   created_t                                   1156720 non-null  int64  
 4   created_datetime                            1156719 non-null  object 
 5   last_modified_t                             1156720 non-null  int64  
 6   last_modified_datetime                      1156720 non-null  object 
 7   product_name                                1101772 non-null  object 
 8   generic_name                                97161 non-null    object 
 9   quantity                                    356992 non-n

Dans l'idée de comparer les produits végans aux produits non-végans, nous allons nous intéresser particulièrement à la composition nutritionnelle de ces produits et à leur nutriscore.

## Nettoyage des données

Voyons le taux de remplissage de chacune des colonnes.

In [4]:
(100*data.count()/data.shape[0]).sort_values(axis=0, ascending=False)

code                                          100.000000
created_t                                     100.000000
states                                        100.000000
states_tags                                   100.000000
states_en                                     100.000000
last_modified_datetime                        100.000000
last_modified_t                               100.000000
url                                           100.000000
created_datetime                               99.999914
creator                                        99.999654
pnns_groups_2                                  99.924528
countries                                      99.834791
countries_tags                                 99.834619
countries_en                                   99.834619
pnns_groups_1                                  98.898869
product_name                                   95.249671
energy_100g                                    79.351269
proteins_100g                  

Beaucoup de features sont très peu renseignées...
Nous allons donc filtrer la base et ne garder que les colonnes qui nous intéressent et qui sont suffisamment remplies.

Commençons par retirer les produits renseignés plusieurs fois, et ayant donc le même code barre.

In [5]:
data['code'].duplicated().value_counts()

False    1156501
True         219
Name: code, dtype: int64

In [6]:
data.drop_duplicates(inplace=True)

Mettons en place une fonction permettant de filtrer notre base de données.

In [7]:
def selection(data, feature, seuil):
    """
        Fonction permettant de filtrer un dataframe
        
        :param data: Dataframe à filtrer
        :param feature: Feature devant être absolument renseignée
        :param seuil: Seuil de remplissage minimum requis pour une feature
        :type data: DataFrame
        :type feature: str
        :type seuil: float
        :return: Dataframe filtré
    """
    my_data = data.dropna(how='any', subset=[feature]).copy(deep=True)
    my_data.dropna(thresh=len(my_data)*seuil, axis=1, inplace=True)
    return my_data

On décide ici de ne garder que les produits où le nutriscore est renseigné et de retirer les features où il manque au moins la moitié des données.

In [8]:
my_data = selection(data, 'nutrition-score-fr_100g', 1/2)

On enlève ensuite les erreurs facilement repérables, i.e. les valeurs supérieures à 100 ou négatives concernant les features "pour 100 grammes", hormis les features "valeur énergétique" où le seuil est fixé à 3800 (100g d'huile d'olive correspond à environ 3600 kcal).

In [9]:
list_100g = my_data.columns[my_data.columns.str.contains('100g')]

err_total=0
for item in list_100g[:-2]: #Les deux derniers éléments sont des nutriscores
    if 'energy' in item:
        errors = my_data[~my_data[item].between(0.01,3800)].index # huile d'olive: environ 3700 kj pour 100g    
        my_data.drop(errors, inplace=True)
        err_total+=errors.size
    else:
        errors = my_data[~my_data[item].between(0,100)].index
        my_data.drop(errors, inplace=True)

print(f'Il y a {err_total} erreurs de ce type, soit {100*err_total/(my_data.shape[0]+err_total):.2f}%')

Il y a 62225 erreurs de ce type, soit 19.00%


On sait également que la somme des quantités de protéines, glucides et lipides, pour 100 grammes d'un produit, ne peut dépasser 100 grammes ni être nulle. On retire donc les produits ne satisfaisant pas ces critères.

In [10]:
errors =  my_data[~(my_data['carbohydrates_100g'] + my_data['proteins_100g'] + my_data['fat_100g']).between(0.01,100)].index
print(f'Il y a {errors.shape[0]} erreurs de ce type, soit {100*errors.shape[0]/my_data.shape[0]:.2f}%')
my_data.drop(errors, inplace=True)

Il y a 1627 erreurs de ce type, soit 0.61%


Dernière chose, "saturated fat" et "sugars" sont en fait des sous-catégories de "fat" et "carbohydrates". On va donc vérifier que la valeur de la sous-catégorie d'un produit n'est pas supérieure à celle de la catégorie parente.
De plus, le sel est composé à 40% de sodium. On va donc s'assurer qu'il y a bien 2,5 fois plus de sel que de sodium dans un produit.

In [11]:
error_sugars = my_data[my_data['carbohydrates_100g']<my_data['sugars_100g']].index
print(f'Il y a {error_sugars.shape[0]} erreurs liées aux sucres, soit {100*error_sugars.shape[0]/my_data.shape[0]:.2f}%')

error_fat = my_data[my_data['fat_100g']<my_data['saturated-fat_100g']].index
print(f'Il y a {error_fat.shape[0]} erreurs liées aux acides gras, soit {100*error_fat.shape[0]/my_data.shape[0]:.2f}%')



sodium = my_data[my_data['sodium_100g']>0]
error_salt = sodium[~(sodium['salt_100g']/sodium['sodium_100g']).between(2.4,2.6)].index
print(f'Il y a {error_salt.shape[0]} erreurs liées au sel, soit {100*error_salt.shape[0]/my_data.shape[0]:.2f}%')

new_errors =  error_fat.union(error_fat).union(error_sugars)
my_data.drop(new_errors, inplace=True)

Il y a 171 erreurs liées aux sucres, soit 0.06%
Il y a 125 erreurs liées aux acides gras, soit 0.05%
Il y a 23 erreurs liées au sel, soit 0.01%


Après observation, on remarque que la feature pnns_groups_1 contient des doublons. On y remédie en supprimant les tirets et les majuscules.

In [12]:
my_data['pnns_groups_1'].value_counts()

Sugary snacks              39419
Milk and dairy products    34681
Fish Meat Eggs             34346
unknown                    32137
Cereals and potatoes       26929
Fruits and vegetables      19755
Composite foods            19582
Fat and sauces             18759
Beverages                  16982
Salty snacks               15577
sugary-snacks               2829
fruits-and-vegetables       1969
cereals-and-potatoes          36
salty-snacks                   4
Name: pnns_groups_1, dtype: int64

In [13]:
my_data['pnns_groups_1'] = my_data['pnns_groups_1'].str.replace('-', ' ').str.lower()

In [14]:
my_data['pnns_groups_1'].value_counts()

sugary snacks              42248
milk and dairy products    34681
fish meat eggs             34346
unknown                    32137
cereals and potatoes       26965
fruits and vegetables      21724
composite foods            19582
fat and sauces             18759
beverages                  16982
salty snacks               15581
Name: pnns_groups_1, dtype: int64

Voyons le taux de remplissage des features conservées.

In [15]:
(100*my_data.count()/my_data.shape[0]).sort_values(axis=0, ascending=False)

nutrition-score-uk_100g                    100.000000
states_en                                  100.000000
url                                        100.000000
creator                                    100.000000
created_t                                  100.000000
created_datetime                           100.000000
last_modified_t                            100.000000
last_modified_datetime                     100.000000
nutriscore_score                           100.000000
nutriscore_grade                           100.000000
nutrition-score-fr_100g                    100.000000
pnns_groups_2                              100.000000
states                                     100.000000
states_tags                                100.000000
code                                       100.000000
energy-kcal_100g                           100.000000
sodium_100g                                100.000000
salt_100g                                  100.000000
proteins_100g               

Pour la suite, le but étant de comparer les produits végans aux produits non végans, nous n'allons garder que les produits dont les ingrédients sont renseignés afin de s'assurer qu'ils soient dans la bonne catégorie.
Enfin, l'appel à projet étant lancé par l'agence "Santé publique France", nous n'allons conserver que les produits disponibles en France.

In [16]:
my_data = my_data[my_data['countries_tags'].str.contains('france', na=False, case=False)]

In [17]:
my_data = selection(my_data, 'ingredients_text', 0)

In [19]:
(100*my_data.count()/my_data.shape[0]).sort_values(axis=0, ascending=False)

nutrition-score-uk_100g                    100.000000
states_tags                                100.000000
pnns_groups_2                              100.000000
nutrition-score-fr_100g                    100.000000
nutriscore_grade                           100.000000
nutriscore_score                           100.000000
ingredients_that_may_be_from_palm_oil_n    100.000000
ingredients_from_palm_oil_n                100.000000
additives_n                                100.000000
ingredients_text                           100.000000
countries_en                               100.000000
countries_tags                             100.000000
countries                                  100.000000
last_modified_datetime                     100.000000
last_modified_t                            100.000000
created_datetime                           100.000000
created_t                                  100.000000
creator                                    100.000000
url                         

Nous allons maintenant ajouter une feature "vegan" à notre jeu de données. Dans cette colonne, un produit considéré comme végan aura comme valeur "True", et "False" dans le cas contraire. 
Pour cela, plusieurs vérifications sont nécessaires, sur les features main_category_en, ingredients_text et pnns_groups_1

In [23]:
my_data['vegan'] = ~(my_data['main_category_en'].str.contains('meat|seafood|dairies|fish|egg|oeuf|farming products|\
terrines|salmon',na= False, case = False) | my_data['ingredients_text'].str.contains('egg|oeuf|huevo|ham|jambon|jamòn|meat|\
viande|carne|cheese|fromage|queso|honey|miel|fish|poisson|pescado|milk|lait|leche|butter|beurre|mantequilla|gelatin|\
gélatine|Schinken|Käse|Schatz|Fisch|Milch', na=False, case=False)| \
my_data['pnns_groups_1'].str.contains('fish|meat',na=False, case=False) |\
my_data['pnns_groups_2'].str.contains('cheese',na=False, case=False))

In [232]:
my_data['vegan'].value_counts()

False    60995
True     44219
Name: vegan, dtype: int64

Pour la suite, nous n'allons conserver ques les features que l'on juge pertinentes pour notre problématique. Deux catégories se distinguent : qualitatives et quantitatives.

In [218]:
features_quanti = ['nutriscore_score', 'salt_100g', 'proteins_100g', 'sugars_100g', 'carbohydrates_100g',\
            'saturated-fat_100g', 'fat_100g', 'energy_100g']
features_quali = ['code', 'brands_tags', 'nutriscore_grade', 'nova_group', 'pnns_groups_1', 'pnns_groups_2',\
                  'main_category_en', 'vegan']

Si une classification n'est pas renseignée, on estime que le produit est dans la plus mauvaise catégorie.

In [ ]:
my_data['nova_group'] = pd.Categorical(my_data['nova_group'], categories=[1.0, 2.0, 3.0, 4.0], ordered=True)
my_data['nova_group'].fillna(4.0)

my_data['nutriscore_grade'] = pd.Categorical(my_data['nutriscore_grade'], categories=list('abcde'), ordered=True)
my_data['nutriscore_grade'].fillna('e')

Pour les autres variables qualitatives, si elles ne sont pas renseignées on les classe en 'unknown'

In [206]:
my_data['brands_tags'].fillna('unknown')
my_data['main_category_en'].fillna('unknown')
my_data['pnns_groups_1'].fillna('unknown') 
my_data['pnns_groups_2'].fillna('unknown')

62                         Bread
371          Sweetened beverages
377        Unsweetened beverages
383                       Fruits
384                       Sweets
                   ...          
1156335       Biscuits and cakes
1156356                  Legumes
1156461                   Cheese
1156639      Sweetened beverages
1156654                   Fruits
Name: pnns_groups_2, Length: 105214, dtype: object

### Sauvergarde des données

In [233]:
my_data[['code','product_name', 'brands_tags', 'nutriscore_grade', 'nova_group', 'pnns_groups_1', 'pnns_groups_2','main_category_en', 'vegan','nutriscore_score', 'salt_100g', 'proteins_100g', 'sugars_100g', 'carbohydrates_100g', 'saturated-fat_100g', 'fat_100g', 'energy_100g']].to_csv(r'my_data.csv', index=False)